In [3]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MyApp") \
    .master("local") \
    .getOrCreate()


In [ ]:
df = spark.read.option('header', 'true').csv('data/customer_orders_report.csv')
df.show()

+-----------+------------------+--------------------+----------+-----------------+
|customer_id|              Name|             Address|last_order|       total_sale|
+-----------+------------------+--------------------+----------+-----------------+
|       9337|        Mary Smith|5687 Lazy Parade,...|2014-07-20|          6585.33|
|       3710|      Ashley Smith|1564 Gentle Jetty...|2014-04-29|           6169.4|
|      10744|    Samantha Smith|5129 Indian Maze,...|2014-06-06|           5799.5|
|        749|    Jesse Matthews|529 Cotton Branch...|2014-07-06|5759.540000000001|
|       5411|      Robert Crane|7723 Lazy Mount,A...|2014-07-07|          5174.56|
|       8314|      Angela Walsh|9922 Round Forest...|2014-05-05|5169.450000000001|
|        173|        Jose Smith|3493 High Apple W...|2014-01-10|5149.370000000001|
|       5186|    Jason Robinson|1965 Shady Robin ...|2014-07-20|          5113.57|
|       7802|      Mary Acevedo|7179 Amber Cider ...|2014-06-25|          4904.45|
|   